**Implicit Euler**
$$
\min_{z\in\mathbb{R}^{51}}\left\|z-y_n-hLz\right\|_{2}^2
$$

$$
\partial_t u = \mathcal{L}u
$$
$$
\dot{u}(t) = Lu(t)\in\mathbb{R}^{51}
$$
$$
\mathbb{R}\ni u_i(t) = \sum_{j=1}^H \alpha_i^j \varphi_j(t),\,\,i=1,...,51
$$
$$
\dot{u}_i(t_c) = \sum_{j=1}^H \alpha_i^j \dot{\varphi}_j(t_c),\,\,c=1,...,C
$$
$$
\dot{u}(t_c) = Lu(t_c)
$$
$$
H_d\alpha_i = (Lu(t))_i,\,\,\alpha_i=[\alpha_i^1,...,\alpha_i^C]\in\mathbb{R}^C,\,H_d\in\mathbb{R}^{C\times H}
$$

Cost linear system euler : $\mathcal{O}(51^3)$
Cost linear system ELM : $\mathcal{O}(51\cdot c)$

where $c$ is the cost of solving the linear system for a single component of the solution

In [279]:
import numpy as np
y0 = np.random.randn(3,1)
oo = np.ones((2,1))
oo@y0.T

array([[-0.38877855, -0.44174312, -0.34074649],
       [-0.38877855, -0.44174312, -0.34074649]])

$$
(I\otimes (hd))
$$

**ELM method**

$$
W\in\mathbb{R}^{H\times 51}
$$
$$
y(t) = y0 + (h-h0)W
$$
$$
h = [\sigma(at_i+b)]_{i=1,...,n}\in\mathbb{R}^{n\times H},\,\,h_0 = \sigma(at_0+b)
$$
$$
W \in\mathbb{R}^{H\times d}
$$

$$
\mathrm{vec}(W)\in\mathbb{R}^{Hd},\,\,H\in\mathbb{R}^{nh},\,\,\mathrm{vec}(Y)\in\mathbb{R}^{dn}
$$

$$
r(t) = F(y(t))-\dot{y}(t) 
$$
$$
\partial_W r(t) = F'(y(t))(h-h0)-(hd)
$$
$$
hd = a\sigma'(at+b)
$$
$$
\frac{\partial \mathrm{vec}(r(t))}{\partial \mathrm{vec}(W)}
$$

$$
\mathrm{vec}(y(t)) = \mathrm{vec}(y0) + (I_d\otimes(h-h0))\mathrm{vec}(W)
$$

$$
\frac{\partial r(t)}{\partial\mathrm{vec}(W)} = F'(y(t))(I\otimes (h-h0)) - \mathrm{vec}(h_d)
$$

$$
F(y) = -y\odot (D_1 y) + D_2 y
$$

$$
F(Y) = -Y\odot (YD_1^T) + YD_2^T
$$
$$
\mathrm{vec}(F(Y)) = -\mathrm{vec}(Y)\odot \mathrm{vec}(YD_1^T)+(D_2\otimes I_n)\mathrm{vec}(Y)
$$

$$
\mathrm{vec}(YD_1^T) = (D_1\otimes I_n)\mathrm{vec}(Y)
$$

$$
\frac{\mathrm{vec}(F(Y))}{\mathrm{vec}(Y)} = - \mathrm{diag}(\mathrm{vec}(YD_1^T)) - \mathrm{diag}(\mathrm{vec}(Y))(D_1\otimes I_n) + (D_2\otimes I_n)
$$

$$
\frac{\partial\mathrm{vec}(Y)}{\partial\mathrm{vec}(W)} = (I_d\otimes (H-H_0))
$$

$$
(D_1\otimes I_n)v = \mathrm{vec}(VD_1),\,\,\mathrm{vec}(V)=v
$$

$H\in\mathbb{R}^{n\times H}$

In [280]:
import numpy as np

v = np.array([1,2.,3])
np.outer(v,np.random.randn(2))

array([[ 0.27738909, -2.03910191],
       [ 0.55477818, -4.07820382],
       [ 0.83216728, -6.11730573]])

(dn,dn) * (dn,dH) = (dn,dH)

$$
\frac{\partial\mathrm{vec}(F(Y))}{\partial\mathrm{vec}(W)} = \frac{\partial\mathrm{vec}(F(Y))}{\partial\mathrm{vec}(Y)}(I_d\otimes (H-H_0))
$$\partial

$$
F'(y) = -\mathrm{diag}(D_1y) - \mathrm{diag}(y)D_1 + D_2
$$

In [281]:
import torch
from torch.func import jacrev
import numpy as np

d = 10
H = 30
n = 20

dtype = torch.float64

Y0 = torch.rand((n,d),dtype=dtype)
h = torch.rand((n,H),dtype=dtype)
h0 = torch.rand((n,H),dtype=dtype)
W = torch.rand((H,d),dtype=dtype)
W.requires_grad =True

D1 = torch.randn((d,d),dtype=dtype)
D2 = torch.randn((d,d),dtype=dtype)

In [282]:
YY = lambda W : Y0 + (h-h0)@W
F = lambda W : - YY(W.reshape(H,d)) * (YY(W.reshape(H,d))@D1.T) + YY(W) @ D2.T

In [283]:
auto_jac = (jacrev(F)(W)).detach().cpu().numpy().reshape((-1,H*d),order='F')

In [284]:
Y = YY(W)

In [285]:
df_dy = - torch.diag_embed((Y@D1.T).T.reshape(-1)) - torch.diag_embed(Y.T.reshape(-1))@torch.kron(D1,torch.eye(n)) + torch.kron(D2,torch.eye(n))
dy_dw = torch.kron(torch.eye(d),h-h0)


In [286]:
df_dw = (df_dy @ dy_dw).detach().cpu().numpy()

In [287]:
np.allclose(df_dw, auto_jac)

True

In [288]:
import numpy as np

d = 2
n = 3
np.kron(np.eye(2),np.random.randn(1,3))

array([[-0.1085951 ,  0.52624065, -0.81295227, -0.        ,  0.        ,
        -0.        ],
       [-0.        ,  0.        , -0.        , -0.1085951 ,  0.52624065,
        -0.81295227]])

### Test for Burgers to make it faster and check it is correct

In [289]:
from scripts.dynamics import vecField
from scripts.utils import act
from scripts.dynamics import vecField

In [290]:
from scipy.sparse import diags, eye, kron
from scipy.sparse.linalg import spsolve
from scipy import sparse

In [291]:
from scipy.optimize import minimize, least_squares

In [292]:
class flowMap:
    def __init__(self,y0,initial_proj,weight,bias,dt=1,n_t=5,n_x=10,L=10,LB=-1.,UB=1.,system="Rober",act_name="Tanh",verbose=False):
        
        self.system = system
        self.vec = vecField(system)
        self.act = lambda t,w,b : act(t,w,b,act_name=act_name)
        self.dt = dt
        self.d = len(y0) #dimension phase space
        
        self.verbose = verbose
        
        self.n_x = n_x
        self.h = np.zeros((n_x,L))
        self.hd = np.zeros((n_x,L))
        
        self.iter = 0
        
        self.y0 = y0
        self.y0_supp = y0
        
        self.L = L #number of neurons
        self.LB = LB #Lower boundary for weight and bias sampling 
        self.UB = UB #Upper boundary for weight and bias sampling
        
        if len(weight)==0:
            self.weight = np.random.uniform(low=self.LB,high=self.UB,size=(self.L))
        else:
            self.weight = weight
        if len(bias)==0:
            self.bias = np.random.uniform(low=self.LB,high=self.UB,size=(self.L))
        else:
            self.bias = bias
        
        self.n_t = n_t
        self.t_tot = np.linspace(0,dt,self.n_t)
        self.x = np.linspace(0,dt,self.n_x)
        
        pattern = np.diag(np.ones(len(y0)))
        vec_up = np.concatenate((np.array([0.]),np.ones(len(y0)-2)))
        vec_down = np.concatenate((np.ones(len(y0)-2),np.array([0.])))
        mat = pattern + np.diag(vec_up,1) + np.diag(vec_down,-1)
        self.sparsity_pattern = np.kron(mat,np.ones((n_x,L)))

        for i in range(n_x):
            self.h[i], self.hd[i] = self.act(self.x[i],self.weight,self.bias)

        self.h0 = self.h[0] #at the initial time, i.e. at x=0.
        self.hf = self.h[-1]
        self.hd0 = self.hd[0]
        self.hdf = self.hd[-1]
        
        self.computational_time = None
        self.computed_projection_matrices = np.tile(initial_proj,(self.n_t-1,1)) #one per time subintreval
        self.computed_initial_conditions = np.zeros((self.n_t-1,self.d)) #one per time subinterval
        self.training_err_vec = np.zeros((self.n_t,1))
        self.sol = np.zeros((self.n_t,self.d))
        
    
    def to_mat(self,w):    
        return w.reshape((self.L,self.d),order='F') #order F means it reshapes by columns
    
    def residual(self,c_i,xi_i):
        y = (self.h-self.h0)@self.to_mat(xi_i) + self.y0_supp.reshape(1,-1)
        vecValue = self.vec.eval(y)
        y_dot = c_i * self.hd @ self.to_mat(xi_i)
        Loss = (y_dot-vecValue)
        return Loss.reshape(-1,order='F')
    
    def re(self,a,H):
        return np.einsum('i,ij->ij',a,H)
    
    def diag_embed(self,x):
        a,b = x.shape
        mat = np.zeros((a,b,b))
        np.einsum('ijj->ij',mat)[:] = x[:]
        return mat #provides a diagonal embedding of a batch of vectors
    
    def jac_residual(self,c_i,xi_i):
        
        y = (np.einsum('jk,ik->ij',self.h-self.h0,self.to_mat(xi_i).T) + self.y0_supp.reshape(-1,1)).T
        H = self.h - self.h0        
        dx, nu, N = self.vec.dx, self.vec.nu, self.vec.N
        n,d = y.shape
        
        vv = np.ones(N-1)
        Shift_forward = diags([vv], [1], shape=(N, N))#np.diag(vv,k=1)
        Shift_backward = diags([vv], [-1], shape=(N, N))#np.diag(vv,k=-1)
        
        Shift_backward = Shift_backward.tolil()
        Shift_backward[-1, -2] = 0  # Adjust for sparse matrix indexing
        Shift_forward = Shift_forward.tolil()
        Shift_forward[0, 1] = 0
        Shift_backward = Shift_backward.tocsr()
        Shift_forward = Shift_forward.tocsr()
        
        #For the boundary conditions
        #Shift_backward[-1]*=0
        #Shift_forward[0]*=0
        
        D2 = (Shift_forward + Shift_backward - 2*np.eye(N))/(dx**2)
        D1 = 1/(2*dx) * (Shift_forward-Shift_backward)
        
        #df_dy = -np.diag((y@D1.T).reshape(-1,order='F')) - np.diag(y.reshape(-1,order='F'))@np.kron(D1,np.eye(n)) + np.kron(D2,np.eye(n))
        df_dy = -sparse.diags((y @ D1.T).reshape(-1, order='F')) - sparse.diags(y.reshape(-1, order='F')) @ kron(D1, eye(n)) + kron(D2, eye(n))
        #dy_dw = np.kron(np.eye(d),H)
        dy_dw = kron(eye(d),H)
        jj = -df_dy@dy_dw
        a = d // n
        #for i in range(a):
        #    jj[i*self.n_x:(i+1)*n,i*n:(i+1)*n] += c_i*self.hd
        return jj.todense()

    def approximate_flow_map(self,IterMax=100,IterTol=1e-5):
        
        self.training_err_vec[0] = 0.        
        self.sol[0] = self.y0_supp
        
        for i in range(self.n_t-1):
            
            self.iter = 1 #Just added
            
            c_i = self.dt / (self.t_tot[i+1]-self.t_tot[i])
            xi_i = self.computed_projection_matrices[i] 
            Loss = self.residual(c_i,xi_i)
            l2 = [2.,1]
            
            self.sparsity_pattern = 1.-1.*(self.jac_residual(c_i,xi_i)==0.)
            
            self.computed_initial_conditions[i] = self.y0_supp
                
            '''while np.abs(l2[1])>IterTol and self.iter<IterMax and np.abs(l2[0]-l2[1])>IterTol:
                
                l2[0] = l2[1] #this says that we suppose to converge after this block of code runs
                JJ = self.jac_residual(c_i,xi_i)
                print((1.-(JJ==0)).sum())
                dxi = np.linalg.lstsq(JJ,Loss,rcond=None)[0]
                xi_i = xi_i - dxi
                Loss = self.residual(c_i,xi_i)
                l2[1] = np.linalg.norm(Loss,ord=2) #This is used as a stopping criterion
                
                self.iter+=1'''
            
            func = lambda x : self.residual(c_i,x)
            print(xi_i.shape)
            xi_i = least_squares(func,x0=self.computed_projection_matrices[i],verbose=2,xtol=1e-2).x#,jac_sparsity=self.sparsity_pattern).x
            '''#Store the computed result
            for i in range(51):
                for j in range(51):
                    print(f"i {i}, j {j}")
                    print(xi_i.jac[i*(self.n_x):(i+1)*(self.n_x),j*(self.L):(j+1)*(self.L)])
            xi_i = xi_i.x'''
            self.computed_projection_matrices[i] = xi_i
            
            y = (np.einsum('jk,ik->ij',self.h-self.h0,xi_i.reshape(len(self.y0),self.L)) + self.y0_supp.reshape(-1,1)).T
            self.y0_supp = y[-1]
            self.sol[i+1] = self.y0_supp
            
            self.training_err_vec[i+1] = np.sqrt(np.mean(Loss**2))

    def analyticalApproximateSolution(self,t):
        j = np.searchsorted(self.t_tot,t,side='left') #determines the index of the largest number in t_tot that is smaller than t
        #In other words, it finds where to place t in t_tot in order to preserve its increasing ordering
          
        y_0 = self.sol[max(0,j-1)]        
        jp = 1 if j==0 else j
        x = np.array([(t - self.t_tot[max(0,j-1)]) / (self.t_tot[jp]-self.t_tot[max(0,j-1)])])
        
        h,_ = act(x,self.weight,self.bias)
        h0,_ = act(0*x,self.weight,self.bias)
        y = (h-h0)@self.to_mat(self.computed_projection_matrices[max(0,j-1)]) + y_0
        return y
    
    def plotOverTimeRange(self,time):
        sol_approximation = np.zeros((self.d,len(time)))
        for i,t in enumerate(time):
            sol_approximation[:,i] = self.analyticalApproximateSolution(t)
        return sol_approximation

In [293]:
system="Burger"
vecRef = vecField(system=system)
nu = vecRef.nu
dx = vecRef.dx
y0 = np.sin(2*np.pi*vecRef.x)
t_max = 1.
L = 50

In [294]:
initial_proj = np.ones((len(y0)*L)).reshape(1,-1)
weight = np.random.randn(L)
bias = np.random.randn(L)
flow = flowMap(y0,initial_proj,weight,bias,system=system,L=L)
flow.approximate_flow_map()

(2550,)
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.9717e+04                                    2.61e+04    
       1              2         3.9302e+03      5.58e+04       5.05e+01       1.29e+04    
       2              3         1.3583e+02      3.79e+03       1.01e+02       1.05e+03    
       3              4         5.7375e+01      7.85e+01       2.02e+02       6.86e+02    
       4              5         2.9881e+00      5.44e+01       4.04e+02       1.56e+02    
       5              6         9.3067e-02      2.90e+00       8.08e+02       2.72e+01    
       6              7         4.4919e-02      4.81e-02       1.62e+03       1.75e+01    
       7              8         1.0437e-02      3.45e-02       3.23e+03       9.42e-01    
       8             10         1.5169e-04      1.03e-02       1.62e+03       1.13e+00    
       9             13         5.6449e-06      1.46e-04       2.02e+02       5.95

### Test LSQR

In [295]:
from scipy.sparse.linalg import lsqr, LinearOperator as linOp
import numpy as np
import time
from scipy.sparse import random,kron

A = np.random.randn(10,1900)
B = np.random.randn(70,200)
b = np.random.randn(700)
Mat = np.kron(A,B)


d1,d2 = A.shape
c1,c2 = B.shape

#x_linalg = np.linalg.solve(Mat,b)

def mv(v):
    V = v.reshape((c2,d2),order='F')
    return (B@V@A.T).reshape((-1,1),order='F')
def rmv(v):
    V = v.reshape((c1,d1),order='F')
    return (B.T@V@A).reshape((-1,1),order='F')
    
M = linOp((c1*d1,c2*d2),matvec=mv,rmatvec=rmv)

tt = time.time()
x_linalg = np.linalg.lstsq(Mat,b,rcond=None)
print("Time for linalg : ",time.time()-tt)

tt = time.time()
x_lsqr = lsqr(M,b,atol=1e-10,btol=1e-10,show=False)[0]
print("Time for lsqr : ",time.time()-tt)
np.allclose(Mat@x_lsqr,b)

### Solving the linear system for Burger's equation

In [ ]:
from scipy.sparse.linalg import lsqr, LinearOperator as linOp
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.sparse import diags, eye, kron
from scipy.sparse.linalg import spsolve
from scipy.sparse import random,kron

N = 51
dx = 1/(N-1)
vv = np.ones(N-1)
Shift_forward = diags([vv], [1], shape=(N, N))#np.diag(vv,k=1)
Shift_backward = diags([vv], [-1], shape=(N, N))#np.diag(vv,k=-1)

Shift_backward = Shift_backward.tolil()
Shift_backward[-1, -2] = 0  # Adjust for sparse matrix indexing
Shift_forward = Shift_forward.tolil()
Shift_forward[0, 1] = 0
Shift_backward = Shift_backward.tocsr()
Shift_forward = Shift_forward.tocsr()

D2 = (Shift_forward + Shift_backward - 2*np.eye(N))/(dx**2)
D1 = 1/(2*dx) * (Shift_forward-Shift_backward)

n_x = 30
L = 10

h = np.zeros((n_x,L))
hd = np.zeros((n_x,L))
LB = -1.
UB = 1.
weight = np.random.uniform(low=LB,high=UB,size=(L))
bias = np.random.uniform(low=LB,high=UB,size=(L))
x = np.linspace(0,1.,n_x)

def sigmoid(x):
    return 1/(1+np.exp(-x))
def act(x,a,b):
    return sigmoid(a*x+b), a*(sigmoid(a*x+b)*(1-sigmoid(a*x+b)**2))
for i in range(n_x):
    h[i], hd[i] = act(x[i],weight,bias)
#h,hd of size n_x x L
h0 = h[0:1]
hd0 = hd[0:1]
domain = np.linspace(0,1,N)
y0 = np.sin(domain*2*np.pi).reshape(-1,1)

W = np.random.randn(L,N)
Y = np.ones((n_x,1))@y0.T + (h-h0)@W
F = - Y * (Y@D1.T) + Y@D2.T
Yd = hd@W

#Dimension of the Jacobian is N n_x x N L
#Thus the input vector v has shape N L 
#and the known vector for the linear system is of N n_x entries
def vec(Y):
    return Y.reshape((-1),order='F')
def to_mat(y,a,b):
    return y.reshape((a,b),order='F')

J = (-np.diag(vec(Y@D1.T))-np.diag(vec(Y))@kron(D1,np.eye(n_x))+kron(D2,np.eye(n_x))) @ np.kron(np.eye(N),(h-h0)) - np.kron(np.eye(N),hd)

print("Conditioning of h-h0 : ",np.linalg.cond(h))
print("Conditioning of the Jacobian matrix : ",np.linalg.cond(J))

def mv(v):
    V = to_mat(v,L,N)
    hV = (h-h0)@V
    return vec(-(Y@D1.T)*hV-Y*((h-h0)@(V@D1.T))+(h-h0)@(V@D2.T)-hd@V)
def rmv(v):
    V = to_mat(v,n_x,N)
    return vec((h-h0).T@[-(Y@D1.T)*V - (Y*V)@D1 + V@D2] - hd.T@V)

v = np.random.randn(L*N)
print("Jacobian vector product : ",np.allclose(mv(v),J@v))
w = np.random.randn(n_x*N)
print("Jacobian transpose vector product : ",np.allclose(rmv(w),J.T@w))

shape = (n_x*N,N*L)
M = linOp(shape,matvec=mv,rmatvec=rmv)

b = np.ones((N*n_x))

tt = time.time()
x_linalg = np.linalg.lstsq(J,b,rcond=None)[0]

print("True shape : ",J.shape)
print("Computed shape  :",shape)
print("xx : ",x_linalg.shape)

print("Time for linalg : ",time.time()-tt)

tt = time.time()
x_lsqr = lsqr(M,b,atol=1e-6,btol=1e-6,show=False)[0]

print("Time for lsqr : ",time.time()-tt)

print("Is lsqr correct? ",np.allclose(J@x_lsqr,b))
print("Is linalg correct? ",np.allclose(J@x_linalg,b))

Conditioning of h-h0 :  163306777647281.56
Conditioning of the Jacobian matrix :  2.3485173553183004e+16
Jacobian vector product :  True
Jacobian transpose vector product :  True
True shape :  (1530, 510)
Computed shape  : (1530, 510)
xx :  (510,)
Time for linalg :  0.04984092712402344
Time for lsqr :  0.6727242469787598
Is lsqr correct?  False
Is linalg correct?  True
